## Project Summary

The human body and its microbiome engage in a dynamic and intricate relationship. The microbiome—particularly the gut microbiota—comprises a vast and metabolically active community, with microbial cells estimated to outnumber human cells by a factor of ten. This ecosystem plays essential roles in digestion, immune function, protection against pathogens, and even brain health. Maintaining a balanced host–microbiota relationship is crucial for overall well-being, as disruptions in this balance—known as dysbiosis—have been linked to a range of conditions, including inflammatory bowel disease (IBD).

This project centers on Crohn’s disease (CD), a chronic inflammatory disorder of the gastrointestinal tract and a major form of IBD. Its global incidence continues to rise, and while its exact cause remains unknown, a combination of genetic predisposition, environmental influences, and microbial imbalance appears to play a role in disease development and progression.

To investigate these microbial contributions, we will use 16S rRNA sequencing on fecal samples from 30 Crohn’s disease patients and 30 healthy controls, all of Ashkenazi Jewish descent from the New York area; publicly availabale dataset provided by UCLA via NCBI's sequnce read archive (SRA). By analyzing differences in gut microbiome composition between the two groups, we aim to identify differentially abundant genera associated with CD. These findings may support the discovery of microbial biomarkers, improve diagnostic strategies, and guide the development of targeted therapies.


In [2]:
import pandas as pd
import os
import subprocess
import qiime2
from qiime2 import Visualization

## Data preprocessing 

We will inspect the metadata and select a subset of the dataset for analysis. We will use 30 diseased and healthy control samples for statistical significance. 

In [2]:
metadata = pd.read_csv("SraRunTable.csv")
metadata.head()

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,BioSampleModel,Bytes,Center Name,collection_date,...,Dysbiosis_Score,ethnicity,ANCA_EU,CBir1_EU,IgA_ASCA_EU,IgG_ASCA_EU,OmpC_EU,Serology_Score,gastrointest_disord,host_body_product
0,SRR27747650,AMPLICON,499,18822579,PRJNA1053656,SAMN39615161,"MIMS.me,MIGS/MIMS/MIMARKS.human-gut",11006343,UCLA,2017-09,...,0.708986,White,13.541820,10.644531,4.396967,3.378769,5.173604,0.240278,Crohn's Disease,NaN
1,SRR27747651,AMPLICON,499,19907717,PRJNA1053656,SAMN39615160,"MIMS.me,MIGS/MIMS/MIMARKS.human-gut",11906253,UCLA,2017-07,...,0.716442,White,15.475319,34.993490,97.820696,63.855626,15.405330,2.179024,Crohn's Disease,NaN
2,SRR27747652,AMPLICON,499,16386433,PRJNA1053656,SAMN39615159,"MIMS.me,MIGS/MIMS/MIMARKS.human-gut",9970643,UCLA,2017-08,...,0.594130,White,5.194735,19.426111,0.724374,0.472128,2.628538,-0.929671,NaN,NaN
3,SRR27747653,AMPLICON,499,20124444,PRJNA1053656,SAMN39615158,"MIMS.me,MIGS/MIMS/MIMARKS.human-gut",12189573,UCLA,2017-08,...,0.584956,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SRR27747654,AMPLICON,499,16590899,PRJNA1053656,SAMN39615157,"MIMS.me,MIGS/MIMS/MIMARKS.human-gut",10134315,UCLA,2017-07,...,0.617854,White,2.993678,30.794271,1.607493,3.874735,5.381472,-0.070397,NaN,NaN


In [3]:
control_samples = metadata[metadata['disease_group'] == 'Control'].head(30) # filter for 30 control groups

In [4]:
cd_samples = metadata[metadata['disease_group'] == 'CD'].head(30) # CD = filter for sample with Crohn's Disease


In [5]:
samples_to_download = pd.concat([cd_samples, control_samples]) # concatenate the filtered disease group and control group datasets

In [7]:
samples_to_download[['Run', 'disease_group', 'host_age', 'host_sex', 'family_id', 'Genetic_Risk_Score', 'Dysbiosis_Score', 'ethnicity']] # Display the filtered Dataframe

,Run,disease_group,host_age,host_sex,family_id,Genetic_Risk_Score,Dysbiosis_Score,ethnicity
0,SRR27747650,CD,35.046575,female,F167,-0.001274,0.708986,White
1,SRR27747651,CD,28.780822,female,F136,-0.002018,0.716442,White
5,SRR27747655,CD,44.104110,male,F102,0.000349,0.739528,White
9,SRR27747659,CD,26.265753,female,F156,-0.002612,NaN,White
14,SRR27747664,CD,32.216438,female,F136,-0.003308,0.900009,White
35,SRR27747685,CD,13.230137,female,F115,-0.003853,0.585463,White
72,SRR27747722,CD,26.657534,male,F198,-0.001305,0.799530,NaN
78,SRR27747728,CD,26.512329,male,F187,-0.002391,0.706355,White
80,SRR27747730,CD,44.824658,male,F135,-0.001400,0.602611,White
91,SRR27747741,CD,20.882192,male,F190,0.000611,0.936082,NaN


In [36]:
output_dir = os.path.expanduser("~/metagenomics/16S_rRNA_fastq")

for run_id in samples_to_download['Run']:
    output_file = os.path.join(output_dir, f"{run_id}.fastq")
    
    
    if os.path.exists(output_file):
        print(f"{run_id} already exists, skipping...")
        continue

    sra_command = f"fastq-dump --outdir {output_dir} {run_id}"

    try:
        subprocess.check_call(sra_command, shell=True)
        print(f"Successfully downloaded {run_id}")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while downloading {run_id}: {e}")

print("Download complete!")


SRR27747650 already exists, skipping...
Read 39823 spots for SRR27747651
Written 39823 spots for SRR27747651
Successfully downloaded SRR27747651
Read 41788 spots for SRR27747655
Written 41788 spots for SRR27747655
Successfully downloaded SRR27747655
Read 12067 spots for SRR27747659
Written 12067 spots for SRR27747659
Successfully downloaded SRR27747659
Read 38666 spots for SRR27747664
Written 38666 spots for SRR27747664
Successfully downloaded SRR27747664
Read 56497 spots for SRR27747685
Written 56497 spots for SRR27747685
Successfully downloaded SRR27747685
Read 41494 spots for SRR27747722
Written 41494 spots for SRR27747722
Successfully downloaded SRR27747722
Read 39694 spots for SRR27747728
Written 39694 spots for SRR27747728
Successfully downloaded SRR27747728
Read 51469 spots for SRR27747730
Written 51469 spots for SRR27747730
Successfully downloaded SRR27747730
Read 43560 spots for SRR27747741
Written 43560 spots for SRR27747741
Successfully downloaded SRR27747741
Read 57528 spot

**Zip the fastq files** 

In [ ]:
!gzip ~/metagenomics/16S_rRNA_fastq/*.fastq

#### Inspecting the fastq files---single/paired-end reads

In [59]:
!zcat SRR27747650.fastq.gz | head -n 10

@SRR27747650.1 M01925:291:GW190114000:1:2105:18051:2021 length=500
TACGTATGGAGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGCAAGGCAAGTCTGATGTGAAAACCCAGGGCTTAACCCTGGGACTGCATTGGAAACTGTCTGGCTCGAGTGCCGGAGAGGTAAGCGGAATTCCTAGTGTAGCGGTGAAATGCGTAGATATTAGGAAGAACACCAGTGGCGAAGGCGGCTTACTGGACGGTAACTGACGTTGAGGCTCGAAAGCGTGGGGAGCAAACCCTGTTTGCTCCCCACGCTTTCGAGCCTCAACGTCAGTTACCGTCCAGTAAGCCGCCTTCGCCACTGGTGTTCTTCCTAATATCTACGCATTTCACCGCTACACTAGGAATTCCGCTTACCTCTCCGGCACTCGAGCCAGACAGTTTCCAATGCAGTCCCAGGGTTAAGCCCTGGGTTTTCACATCAGACTTGCCTTGCCGTCTACGCTCCCTTTACACCCAGTAAATCCGGATAACGCTTGCTCCATAC
+SRR27747650.1 M01925:291:GW190114000:1:2105:18051:2021 length=500
BBCBAFFFFFFFCGGGFGGGGGHGGGGGHHGHHGDBEGHHHHHGHGGGGGGGHGGGGEFGHHHGHFHHHHHHHHHHFFEFEHGGEHHHHHHHHHHGHHHHHHHHHHHHHHHHHHGHBGHHGGGHCHHGG//>AGHHHHHGGGGGHHHHGHHHHHHHGGGDGHHGHHGGGGFHHHHHGGHHGFFGGGGGGGGGGGGGGGGGFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFDDFD.ABFAFFFEFF?BBBAAFFFFFFFGGGGCFGGGGEFHGGHHHFHFHHGGHHHHHGGGGGHHHHHHGFGGGGHFFEGGHGHGHHGHHHGHHGHHHHHHHGGGGGGHHHHHGEGGGGHHHHGHHHHHHG

#### Import qiime artifact

**The sequence format we see above are either single-end reads or fused (forward and reverse) with 250 bp for each sides. We will use single-read format to import qiime2 artifacts**

In [12]:
!qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path ~/metagenomics/16S_rRNA_fastq/manifest.tsv \
  --input-format SingleEndFastqManifestPhred33V2 \
  --output-path demux-single-end.qza


Imported /home/tadi/metagenomics/16S_rRNA_fastq/manifest.tsv as SingleEndFastqManifestPhred33V2 to demux-single-end.qza


In [13]:
!qiime demux summarize \
  --i-data demux-single-end.qza \
  --o-visualization single-end-demux.qzv

Saved Visualization to: single-end-demux.qzv


In [6]:
# Load the .qzv visualization file and display
viz = qiime2.Visualization.load('single-end-demux.qzv')
viz

<visualization: Visualization uuid: c50bcc62-ed18-4119-a791-9100e8df8415>

 ## Quality control: use dada2 to denoise and generate ASVs
 
 We will use dada2 to denoise and generate the amplicon sequence variants (ASV) files. Based on the quality plot above we will truncate at read length of 150 and also filter-out reads with quality score below 20.

In [15]:
!qiime dada2 denoise-single \
  --i-demultiplexed-seqs demux-single-end.qza \
  --p-trunc-len 150 \
  --p-trim-left 0 \
  --p-trunc-q 20 \
  --o-table table.qza \
  --o-representative-sequences rep-seqs.qza \
  --o-denoising-stats denoising-stats.qza


Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza
Saved SampleData[DADA2Stats] to: denoising-stats.qza


### Visualize the quality of the denoised result

In [16]:
!qiime metadata tabulate \
  --m-input-file denoising-stats.qza \
  --o-visualization denoising-stats.qzv


Saved Visualization to: denoising-stats.qzv


In [7]:
# visualization of the denoised file 
viz = qiime2.Visualization.load('denoising-stats.qzv')
viz

<visualization: Visualization uuid: 095a091e-846e-4614-ae29-76ae2f917fbb>

In [19]:
!qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv


Saved Visualization to: table.qzv


#### Visualize the feature-table

In [3]:
viz = qiime2.Visualization.load('table.qzv')
viz

<visualization: Visualization uuid: 458081fd-5e35-46cc-8ff5-4ba742e40327>

**Generate the representative sequences**

In [22]:
!qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv


Saved Visualization to: rep-seqs.qzv


#### Visualiaz the rep-seqs

In [12]:

viz = qiime2.Visualization.load('rep-seqs.qzv')
viz


<visualization: Visualization uuid: 912196e7-453c-40ed-9f62-694f724c9a8f>

## Taxonomy Assignment: using pre-trained reference Silva classifier

I used qiime2-2024.5 version for this analysis and one can get a pre-trained Silva classifier that matches qiime2-2024.5 using the command @ !wget https://data.qiime2.org/classifiers/sklearn-1.4.2/silva/silva-138-99-nb-classifier.qza

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier silva-138-99-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza
  


In [27]:
!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

Saved Visualization to: taxonomy.qzv


In [10]:
# Load and display the taxons
viz = qiime2.Visualization.load('taxonomy.qzv')
viz

<visualization: Visualization uuid: 712ba594-52b3-494d-8590-90b5f8b2ba6b>

*Note that recent Silva classifiers like the one used here have depracated species level resolution for 16S rRNA samples (too similar at the species level to even detect meaningfull difference); as that leads to false positives!* The taxa bar plot will help us assess relative abundance of microbial taxon between samples or accross groups like CD/healthy. The colors within the bars corresponds to different taxonomic groups helping us identify dominant taxa and detect shifts in microbiota associated with disease states or other variables. 

In [33]:
!qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization taxa-barplot.qzv


Saved Visualization to: taxa-barplot.qzv


In [15]:
viz = qiime2.Visualization.load('taxa-barplot.qzv')
viz

<visualization: Visualization uuid: c74b8446-95c0-49ae-90bc-77ab39b90cdc>

## Diversity Analysis: alpha and beta diversity


In [35]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-alignment.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-alignment.qza
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza


**Based on the feature-table summary (table.qzv) above, we will Choose a rarefaction depth at 16,000 (1st quartile, 75% retention) for meaningfull diversity analysis** 

In [ ]:
!qiime feature-table rarefy \
  --i-table table.qza \
  --p-sampling-depth 16000 \
  --o-rarefied-table rarefied-table.qza


**Compute Alpha Diversity**: we will use both faith-pd and shannon index to compute within sample diversity.

In [37]:
!qiime diversity alpha-phylogenetic \
  --i-table rarefied-table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-metric faith_pd \
  --o-alpha-diversity faith-pd-alpha-diversity-results.qza

Saved SampleData[AlphaDiversity] to: faith-pd-alpha-diversity-results.qza


In [38]:
!qiime metadata tabulate \
  --m-input-file faith-pd-alpha-diversity-results.qza \
  --o-visualization faith-pd-summary.qzv

/home/tadi/miniconda3/envs/qiime2-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
Saved Visualization to: faith-pd-summary.qzv


In [17]:
viz = qiime2.Visualization.load('faith-pd-summary.qzv')
viz

<visualization: Visualization uuid: be3dd79b-87ef-4e20-8c5d-fc427c961e2d>

 **note that after rarefaction we lost 10 samples; i.e, two controls and 8 CD samples altogether**

#### Using the Faith-pd metric to calculate alpha-group-significance 

**We will compute the alpha group significance i.e., microbial distribution difference between the two groups**

In [40]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity faith-pd-alpha-diversity-results.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization faith-pd-group-significance.qzv

Saved Visualization to: faith-pd-group-significance.qzv


In [18]:
viz = qiime2.Visualization.load('faith-pd-group-significance.qzv')
viz

<visualization: Visualization uuid: 43bfbff5-7496-4ada-aa3f-1837535ef2bb>

#### Using the shannon index to calculate alpha-group-significance 

In [47]:
!qiime diversity alpha \
  --i-table rarefied-table.qza \
  --p-metric shannon \
  --o-alpha-diversity shannon-alpha-diversity.qza


Saved SampleData[AlphaDiversity] to: shannon-alpha-diversity.qza


In [48]:
!qiime metadata tabulate \
  --m-input-file shannon-alpha-diversity.qza \
  --o-visualization shannon-alpha-diversity.qzv


/home/tadi/miniconda3/envs/qiime2-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
Saved Visualization to: shannon-alpha-diversity.qzv


In [50]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity shannon-alpha-diversity.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization shannon-group-significance.qzv

Saved Visualization to: shannon-group-significance.qzv


In [4]:
viz = qiime2.Visualization.load('shannon-group-significance.qzv')
viz

<visualization: Visualization uuid: 922a5c89-72a1-414d-9bd7-08cc7df076e7>

**Report on the Alpha Diversity Analysis**

Both Faith’s Phylogenetic Diversity and the Shannon index were significantly lower in Crohn’s disease patients compared to healthy controls (p < 0.01), indicating a consistent reduction in gut microbial diversity.

Faith’s PD captures a loss of phylogenetic richness, suggesting a narrower evolutionary spread of microbes in CD, while the Shannon index reflects reduced taxonomic richness and/or evenness, pointing to fewer or more unevenly distributed taxa.

Together, these results support the presence of microbial dysbiosis in Crohn’s disease, with CD samples showing a visibly compressed diversity range relative to controls, as seen in the box plots. This aligns with the hypothesis that gut microbial diversity is significantly altered in Crohn’s disease.

## Compute beta diversity using unweighted unifrac 

In [52]:
!qiime diversity beta-phylogenetic \
  --i-table rarefied-table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-metric unweighted_unifrac \
  --o-distance-matrix unweighted-unifrac-distance.qza



Saved DistanceMatrix to: unweighted-unifrac-distance.qza


In [53]:
!qiime diversity pcoa \
  --i-distance-matrix unweighted-unifrac-distance.qza \
  --o-pcoa unweighted-unifrac-pcoa.qza

Saved PCoAResults to: unweighted-unifrac-pcoa.qza


In [54]:
!qiime emperor plot \
  --i-pcoa unweighted-unifrac-pcoa.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization unweighted-unifrac-emperor.qzv

Saved Visualization to: unweighted-unifrac-emperor.qzv


In [19]:
viz = qiime2.Visualization.load('unweighted-unifrac-emperor.qzv')
viz

<visualization: Visualization uuid: 0adb2672-765f-4578-adbd-ffc23eb81afb>

**Use PERMANOVA to test statistical significance between groups**

In [59]:
!qiime diversity beta-group-significance \
  --i-distance-matrix unweighted-unifrac-distance.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column disease_group \
  --o-visualization permanova-unifrac.qzv \
  --p-pairwise

Saved Visualization to: permanova-unifrac.qzv


In [20]:
viz = qiime2.Visualization.load('permanova-unifrac.qzv')
viz

<visualization: Visualization uuid: d70817ad-0d92-438b-8b5d-c04fdc9a19aa>

#### Report on the beta diversity group significance

We assessed beta diversity using UniFrac distance metrics, followed by Principal Coordinates Analysis (PCoA) to visualize the variation in microbial community composition across samples. The PCoA plot showed clear separation between CD (red) and healthy control samples (blue), with most samples clustering by group. While 5 control samples appeared within the CD cluster, suggesting minor mixing likely reflecting inter-individual variability, the overall distribution supports a distinct microbial community structure associated with CD.

To test the statistical significance of this separation, we performed a PERMANOVA. The result was significant (pseudo-F = 5.05, p = 0.001, q = 0.001), indicating that the differences in microbiome composition between CD and control groups are unlikely to be due to chance.

These findings provide robust support for the hypothesis that Crohn’s disease is associated with a shift in gut microbial community structure, consistent with microbial dysbiosis.

## Differential abundance at the genus level

As we are using 16S rRNA which are good for species level resolution, we will collapse the feature table to genera level.

In [9]:
!qiime taxa collapse \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table table-genus.qza


Saved FeatureTable[Frequency] to: table-genus.qza


In [10]:
!qiime composition add-pseudocount \
  --i-table table-genus.qza \
  --o-composition-table comp-table-genus.qza

!qiime composition ancom \
  --i-table comp-table-genus.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column disease_group \
  --o-visualization ancom-genus.qzv


Saved FeatureTable[Composition] to: comp-table-genus.qza
Saved Visualization to: ancom-genus.qzv


In [8]:
viz = qiime2.Visualization.load('ancom-genus.qzv')
viz

<visualization: Visualization uuid: 54d46913-7224-41d5-938c-720cb7f2e321>

**Report on the differential abundance of ancom results**

Differential abundance analysis using ANCOM identified two genera—Flavonifractor and Ruminococcus_gnavus_group as significantly enriched in individuals with Crohn’s disease (CD) compared to healthy controls. Both taxa are members of the phylum Firmicutes and have been implicated in gut microbiota dysbiosis. Notably, Flavonifractor has been linked to the degradation of dietary flavonoids and pro-inflammatory activity, while Ruminococcus_gnavus_group is known to produce inflammatory polysaccharides and is frequently elevated in IBD patients. Their enrichment in CD samples aligns with previous findings and supports their potential involvement in promoting intestinal inflammation and disease severity. These results contribute to the growing evidence of microbiome shifts associated with CD pathogenesis.